In [5]:
import networkx as nx
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter
import itertools
import requests
from io import StringIO

In [7]:
# source for the most populated cities: https://worldpopulationreview.com/cities
def get_city_data():
    city_url = 'https://worldpopulationreview.com/cities'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(city_url, headers=headers)
        response.raise_for_status()
        tables = pd.read_html(StringIO(response.text))
        df_cities = tables[0]
        cities = df_cities['City'].str.split(',').str[0].tolist()
        print(f"Successfully scraped {len(df_cities)} cities.")
        return cities[:500]
    except requests.exceptions.RequestException as e:
        print(f"An error occurred during scraping: {e}")
    except IndexError:
        print("Could not find the expected table on the page.")
    return []

cities = get_city_data()
if cities:
    print(f"Using the top {len(cities)} most populated cities for the graph.")

Successfully scraped 822 cities.
Using the top 500 most populated cities for the graph.


In [8]:
# Create the directed graph for the cities
city_atlas_graph = nx.DiGraph()
city_atlas_graph.add_nodes_from(cities)

city_edges = [
    (c1, c2) for c1, c2 in itertools.product(cities, repeat=2)
    if c1 != c2 and c1.lower()[-1] == c2.lower()[0]
]
city_atlas_graph.add_edges_from(city_edges)

print(f"City graph created with {city_atlas_graph.number_of_nodes()} cities and {city_atlas_graph.number_of_edges()} possible moves.")

City graph created with 498 cities and 9209 possible moves.


In [15]:
def create_clean_interactive_graph(G, graph_type="Items"):
    pos = nx.kamada_kawai_layout(G)


    edge_x, edge_y = [], []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.4, color='#888'),
        hoverinfo='none',
        mode='lines',
        visible='legendonly',
        name='Connections'
    )

    node_x, node_y, node_hovertext, node_color, node_size = [], [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        in_degree = G.in_degree(node)
        out_degree = G.out_degree(node)

        node_x.append(x)
        node_y.append(y)

        hovertext = (
            f"<b>{node}</b><br>"
            f"Options From Here: {out_degree}<br>"
            f"Paths To Here: {in_degree}<br>"
            f"Strategic Value: {out_degree - in_degree}"
        )
        node_hovertext.append(hovertext)
        node_color.append(out_degree - in_degree)
        node_size.append(5 + (in_degree + out_degree) / 2)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        text=node_hovertext,
        marker=dict(
            showscale=True,
            colorscale='RdYlGn',
            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Strategic Value',
                xanchor='left',
                titleside='right'
            ),
            line_width=1,
            line_color='black'
        ),
        name=graph_type
    )

    fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title=f'Atlas Game: Strategic Map of {graph_type}',
                titlefont_size=20,
                showlegend=True,
                legend_title_text='Toggle View',
                legend=dict(
                    x=0.01,
                    y=0.99,
                    xanchor='left',
                    yanchor='top'
                ),
                hovermode='closest',
                margin=dict(b=5, l=5, r=5, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                plot_bgcolor='white',
                height=700,
                width=1000
            )
    )
    fig.show()

def perform_graph_analysis(G):
    analysis = {}
    analysis['in_degrees'] = dict(G.in_degree)
    analysis['out_degrees'] = dict(G.out_degree)
    analysis['dead_ends'] = [n for n, d in analysis['out_degrees'].items() if d == 0]
    analysis['pagerank'] = nx.pagerank(G)
    analysis['betweenness'] = nx.betweenness_centrality(G)
    start_letters = Counter(c[0].lower() for c in G.nodes())
    end_letters = Counter(c[-1].lower() for c in G.nodes())
    all_letters = sorted(list(set(start_letters.keys()) | set(end_letters.keys())))
    analysis['letter_advantage'] = {l: start_letters.get(l, 0) - end_letters.get(l, 0) for l in all_letters}
    analysis['start_letters'] = start_letters
    analysis['end_letters'] = end_letters
    analysis['all_letters'] = all_letters
    return analysis

def create_analysis_dashboard(results, graph_type="Items"):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=("Options After Naming (Out-Degree)", "How Often an Item is an Option (In-Degree)",
                        "Starting vs. Ending Letter Frequency", "Strategic Letter Advantage"),
        vertical_spacing=0.15, horizontal_spacing=0.05
    )
    fig.add_trace(go.Histogram(x=list(results['out_degrees'].values()), name='Out-Degree'), row=1, col=1)
    fig.add_trace(go.Histogram(x=list(results['in_degrees'].values()), name='In-Degree'), row=1, col=2)
    start_counts = [results['start_letters'].get(l, 0) for l in results['all_letters']]
    end_counts = [results['end_letters'].get(l, 0) for l in results['all_letters']]
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=start_counts, name='Starts With'), row=2, col=1)
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=end_counts, name='Ends With'), row=2, col=1)
    advantages = list(results['letter_advantage'].values())
    colors = ['green' if adv > 0 else 'red' if adv < 0 else 'grey' for adv in advantages]
    fig.add_trace(go.Bar(x=[l.upper() for l in results['all_letters']], y=advantages, marker_color=colors, name='Advantage'), row=2, col=2)
    fig.update_layout(height=800, width=1200, title_text=f"Atlas Game: Strategic Analysis Dashboard for {graph_type}",
                      barmode='group', showlegend=False)
    fig.show()

class AtlasGameAdvisor:
    def __init__(self, graph, analysis_results):
        self.graph = graph
        self.analysis = analysis_results
        self.used_items = set()
        self.item_scores = self._calculate_scores()

    def _calculate_scores(self):
        scores = {}
        for item in self.graph.nodes():
            out_deg = self.analysis['out_degrees'].get(item, 0)
            pr = self.analysis['pagerank'].get(item, 0)
            betweenness = self.analysis['betweenness'].get(item, 0)
            score = (out_deg * 2) + (pr * 500) + (betweenness * 200)
            if out_deg == 0:
                score -= 100
            scores[item] = score
        return scores

    def recommend_moves(self, last_item):
        valid_moves = [s for s in self.graph.successors(last_item) if s not in self.used_items]
        if not valid_moves:
            return "Opponent is trapped! You win!"
        return sorted([(move, self.item_scores.get(move, 0)) for move in valid_moves], key=lambda x: x[1], reverse=True)

    def mark_used(self, item):
        self.used_items.add(item)

def start_interactive_session(advisor, graph_nodes):
    print("\n" + "="*60 + "\nATLAS GAME STRATEGY ADVISOR (CITIES)\n" + "="*60)
    print("Enter opponent's city to get strategic advice. Commands: 'reset', 'quit'")

    while True:
        user_input = input("\n> Opponent played: ").strip()
        if user_input.lower() == 'quit': break
        if user_input.lower() == 'reset':
            advisor.used_items.clear()
            print("Game has been reset.")
            continue

        matches = [c for c in graph_nodes if c.lower().startswith(user_input.lower())]
        if not matches:
            print(f"Sorry, couldn't find a city starting with '{user_input}'.")
            continue

        selected_item = matches[0]
        advisor.mark_used(selected_item)
        print(f"Opponent's move '{selected_item}' marked as used.")

        recommendations = advisor.recommend_moves(selected_item)

        if isinstance(recommendations, str):
            print(f"\n{recommendations}"); break

        print("\n--- Recommended Moves (Best to Worst) ---")
        for i, (move, score) in enumerate(recommendations[:5]):
            dead_end_tag = " (WINNING TRAP!)" if advisor.analysis['out_degrees'].get(move, 0) == 0 else ""
            print(f"{i+1}. {move}{dead_end_tag}")

        if recommendations:
            best_move = recommendations[0][0]
            advisor.mark_used(best_move)
            print(f"\nAuto-played best move '{best_move}' and marked as used.")

In [16]:
create_clean_interactive_graph(city_atlas_graph, "Cities")

In [17]:
if cities:

    city_analysis_results = perform_graph_analysis(city_atlas_graph)

    create_analysis_dashboard(city_analysis_results, "Cities")

    city_advisor = AtlasGameAdvisor(city_atlas_graph, city_analysis_results)


In [13]:
start_interactive_session(city_advisor, list(city_atlas_graph.nodes()))


ATLAS GAME STRATEGY ADVISOR (CITIES)
Enter opponent's city to get strategic advice. Commands: 'reset', 'quit'

> Opponent played: Delhi
Opponent's move 'Delhi' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Istanbul
2. Islamabad
3. Ibadan
4. Incheon
5. Izmir

Auto-played best move 'Istanbul' and marked as used.

> Opponent played: London
Opponent's move 'London' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Naples
2. Nagoya
3. N-Djamena
4. Nouakchott
5. Nashik

Auto-played best move 'Naples' and marked as used.

> Opponent played: Sydney
Opponent's move 'Sydney' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Yangon
2. Yinchuan
3. Yichang
4. Yekaterinburg
5. Yueqing

Auto-played best move 'Yangon' and marked as used.

> Opponent played: Noida
Sorry, couldn't find a city starting with 'Noida'.

> Opponent played: Nagpur
Opponent's move 'Nagpur' marked as used.

--- Recommended Moves (Best to Worst) ---
1. Rajkot
2. Rabat
3. Riyadh
4. Ruian
5